Migrate SQL Server Instance to Azure SQL Server VM
================================

### Load Required Modules


In [2]:
Import-Module dbatools
Import-Module Az.Resources
Import-Module Az.Storage

### Choose Migration Source

Required parameters:

- Server Name
- Database Name
- 

In [4]:
$Credential = Get-Credential -Message "Type the name and password of the local administrator account."
$Credential

In [18]:
# Variables

## Global
$Location = "West US 2"
$ResourceGroupName = "sqlmig"

## Storage
$StorageName = $ResourceGroupName + "storage"
$StorageSku = "Premium_LRS"

## Network
$InterfaceName = $ResourceGroupName + "ServerInterface"
$NsgName = $ResourceGroupName + "nsg"
$VNetName = $ResourceGroupName + "VNet"
$SubnetName = "Default"
$VNetAddressPrefix = "10.0.0.0/16"
$VNetSubnetAddressPrefix = "10.0.0.0/24"
$TCPIPAllocationMethod = "Dynamic"
$DomainName = $ResourceGroupName

##Compute
$VMName = $ResourceGroupName + "VM"
$ComputerName = $ResourceGroupName + "Server"
$VMSize = "Standard_DS13_v2"
$OSDiskName = $VMName + "OSDisk"

##Image
$PublisherName = "MicrosoftSQLServer"
$OfferName = "SQL2017-WS2016"
$Sku = "SQLDEV"
$Version = "latest"


In [19]:
# Resource Group
New-AzResourceGroup -Name $ResourceGroupName -Location $Location
$ResourceGroupName
$Location



ResourceGroupName : sqlmig
Location          : westus2
ProvisioningState : Succeeded
Tags              : 
ResourceId        : /subscriptions/172ee93a-2450-4468-aa4d-1a2c4b249ccd/resourceGroups/sqlmig

sqlmig
West US 2




In [21]:
# Storage
$StorageAccount = New-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name $StorageName -SkuName $StorageSku -Kind "Storage" -Location $Location


New-AzStorageAccount : The storage account named sqlmigstorage is already taken.
Parameter name: Name
At line:3 char:19
+ ... geAccount = New-AzStorageAccount -ResourceGroupName $ResourceGroupNam ...
+                 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : CloseError: (:) [New-AzStorageAccount], ArgumentException
    + FullyQualifiedErrorId : Microsoft.Azure.Commands.Management.Storage.NewAzureStorageAccountCommand
 
Get-AzStorageAccount : Cannot process command because of one or more missing mandatory parameters: ResourceGroupName.
At line:4 char:1
+ Get-AzStorageAccount -Name $StorageName
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidArgument: (:) [Get-AzStorageAccount], ParameterBindingException
    + FullyQualifiedErrorId : MissingMandatoryParameter,Microsoft.Azure.Commands.Management.Storage.GetAzureStorageAcc 
   ountCommand
 


In [27]:
$StorageAccount = Get-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name $StorageName

In [22]:
# Network
$SubnetConfig = New-AzVirtualNetworkSubnetConfig -Name $SubnetName -AddressPrefix $VNetSubnetAddressPrefix
$VNet = New-AzVirtualNetwork -Name $VNetName -ResourceGroupName $ResourceGroupName -Location $Location -AddressPrefix $VNetAddressPrefix -Subnet $SubnetConfig
$PublicIp = New-AzPublicIpAddress -Name $InterfaceName -ResourceGroupName $ResourceGroupName -Location $Location -AllocationMethod $TCPIPAllocationMethod -DomainNameLabel $DomainName
#$NsgRuleRDP = New-AzNetworkSecurityRuleConfig -Name "RDPRule" -Protocol Tcp -Direction Inbound -Priority 1000 -SourceAddressPrefix * -SourcePortRange * -DestinationAddressPrefix * -DestinationPortRange 3389 -Access Allow
$NsgRuleSQL = New-AzNetworkSecurityRuleConfig -Name "MSSQLRule"  -Protocol Tcp -Direction Inbound -Priority 1001 -SourceAddressPrefix * -SourcePortRange * -DestinationAddressPrefix * -DestinationPortRange 1433 -Access Allow
#$Nsg = New-AzNetworkSecurityGroup -ResourceGroupName $ResourceGroupName -Location $Location -Name $NsgName -SecurityRules $NsgRuleRDP,$NsgRuleSQL
$Nsg = New-AzNetworkSecurityGroup -ResourceGroupName $ResourceGroupName -Location $Location -Name $NsgName -SecurityRules $NsgRuleSQL
$Interface = New-AzNetworkInterface -Name $InterfaceName -ResourceGroupName $ResourceGroupName -Location $Location -SubnetId $VNet.Subnets[0].Id -PublicIpAddressId $PublicIp.Id -NetworkSecurityGroupId $Nsg.Id



 Old Way : -ResourceId
 New Way : -NatGatewayId

 Old Way : -InputObject
 New Way : -NatGateway
information on breaking changes in Azure PowerShell.


In [28]:
$StorageAccount


StorageAccountName ResourceGroupName PrimaryLocation SkuName     Kind    AccessTier CreationTime         ProvisioningSt
                                                                                                         ate           
------------------ ----------------- --------------- -------     ----    ---------- ------------         --------------
sqlmigstorage      sqlmig            westus2         Premium_LRS Storage            1/10/2020 1:10:46 AM Succeeded     




In [33]:
# Compute
$VirtualMachine = New-AzVMConfig -VMName $VMName -VMSize $VMSize
#$Credential = Get-Credential -Message "Type the name and password of the local administrator account."
$VirtualMachine = Set-AzVMOperatingSystem -VM $VirtualMachine -Windows -ComputerName $ComputerName -Credential $Credential -ProvisionVMAgent -EnableAutoUpdate #-TimeZone = $TimeZone
$VirtualMachine = Add-AzVMNetworkInterface -VM $VirtualMachine -Id $Interface.Id
#$OSDiskUri = $StorageAccount.PrimaryEndpoints.Blob.ToString() + "vhds/" + $OSDiskName + ".vhd"
#$VirtualMachine = Set-AzVMOSDisk -VM $VirtualMachine -Name $OSDiskName -VhdUri $OSDiskUri -Caching ReadOnly -CreateOption FromImage
$VirtualMachine = Set-AzVMSourceImage -VM $VirtualMachine -PublisherName $PublisherName -Offer $OfferName -Skus $Sku -Version $Version

In [34]:
# Create the VM in Azure
New-AzVM -ResourceGroupName $ResourceGroupName -Location $Location -VM $VirtualMachine


cliensqlmigsqlm010917170, is used for boot diagnostics.

RequestId IsSuccessStatusCode StatusCode ReasonPhrase
--------- ------------------- ---------- ------------
                         True         OK OK          




In [35]:
# Add the SQL IaaS Extension, and choose the license type
New-AzSqlVM -ResourceGroupName $ResourceGroupName -Name $VMName -Location $Location -LicenseType "PAYG"


Name     ResourceGroupName LicenseType Sku       Offer          SqlManagementType
----     ----------------- ----------- ---       -----          -----------------
sqlmigVM sqlmig            PAYG        Developer SQL2017-WS2016 LightWeight      




### Verify No Active Connections

In [13]:

$ServerName = "sqltools2017-3"
$DatabaseName = "Keep_WideWorldImporters"

Get-DbaProcess -SqlInstance $ServerName -Database $DatabaseName | 
Select Host, login, Program


Host Login Program
---- ----- -------
     sa           




### Create Target SQL Server VM

### Create temporary resources for data movement

In [0]:
$location = "westus"
$resourceGroup = "temp-sqlmigration"
$blobStorageAccount = "temp-sqlmigration"
$containerName = "backups"



New-AzResourceGroup

In [14]:
$targetLogin = Get-Credential -Message "Login to target SQL Server instance as:"

In [1]:
$myGwIp = Get-AzPublicIpAddress -Name $GwIP1 -ResourceGroup $RG1
$myGwIp.IpAddress

Get-AzPublicIpAddress : Cannot validate argument on parameter 'Name'. The argument is null or empty. Provide an 
argument that is not null or empty, and then try the command again.
At line:2 char:39
+ $myGwIp = Get-AzPublicIpAddress -Name $GwIP1 -ResourceGroup $RG1
+                                       ~~~~~~
    + CategoryInfo          : InvalidData: (:) [Get-AzPublicIpAddress], ParameterBindingValidationException
    + FullyQualifiedErrorId : ParameterArgumentValidationError,Microsoft.Azure.Commands.Network.GetAzurePublicIpAddres 
   sCommand
 
